In [148]:
import pandas as pd
import os
import sys
import numpy as np
import openmatrix as omx
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from TransportationNetworks import Network 
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.misc import derivative
import dask
from dask.diagnostics import ProgressBar
import ipycytoscape

In [ ]:
#https://towardsdatascience.com/parallel-processing-with-python-and-aws-ec2-3b01fdc8272

In [150]:
# Assume you have not changed the relative paths after cloning the repository
root = os.path.dirname(os.path.abspath('.')) + '/Storied-Streets-of-Barcelona'+ '/TransportationNetworks-master'
root

'/Users/erikagutierrez/Documents/BSE/Term_2/Networks/Assignments/Final Project/Storied-Streets-of-Barcelona/TransportationNetworks-master'

In [151]:
#paths to load files
city = 'Philadelphia'
directory = root + "/{}/".format(city)
link_file = '{}{}_net.tntp'.format(directory, city)
trip_file = '{}{}_trips.tntp'.format(directory, city)
node_file = '{}{}_node.tntp'.format(directory, city)

In [152]:
net = pd.read_csv(link_file, skiprows=8, sep='\t')
trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed
net.drop(['~', ';'], axis=1, inplace=True)

In [153]:
# Define the treatment variable and covariates
net['has_toll'] = 0

for i in range(len(net)):
    if net.loc[i, 'toll'] != 0:
        net.loc[i, 'has_toll'] = 1   

In [154]:
# Fit a logistic regression model to obtain propensity scores
covariates = ['capacity', 'length', 'free_flow_time', 'b', 'power', 'speed', 'link_type']
log_reg = LogisticRegression()
X = net[covariates]
y = net['has_toll']
log_reg.fit(X, y)
net['prop_score'] = model.predict_proba(X)[:, 1]

In [155]:
# Prep dataset for Linear Regression

control = net[net['has_toll'] == 0].sort_values('prop_score', ascending = False)[:32]
treatment = net[net['has_toll'] == 1]

ols_df = pd.concat([control, treatment])

In [156]:
# fit Linear Regression to obtain effect of toll on free flow time
covariates = ['capacity', 'length', 'has_toll', 'b', 'power', 'speed', 'link_type']
lin_reg = LinearRegression()
X = ols_df[covariates]
y = ols_df['free_flow_time']
lin_reg.fit(X, y)
lin_reg.coef_

array([-2.86469798e-06,  8.55097238e-01, -3.26724640e-01,  2.70006240e-13,
        0.00000000e+00, -3.53529067e+02,  1.79381326e-01])

In [157]:
SO = True
Philly = Network(link_file, trip_file,node_file, SO)

['1 30208 74789\n']
['2 30224 74793\n']
['3 30247 74783\n']
['4 30220 74767\n']
['5 30243 74752\n']
['6 30227 74755\n']
['7 30211 74759\n']
['8 30200 74760\n']
['9 30180 74755\n']
['10 30173 74765\n']
['11 30183 74773\n']
['12 30178 74793\n']
['13 30153 74763\n']
['14 30158 74777\n']
['15 30151 74790\n']
['16 30275 74793\n']
['17 30291 74777\n']
['18 30285 74743\n']
['19 30268 74746\n']
['20 30264 74723\n']
['21 30248 74733\n']
['22 30224 74738\n']
['23 30230 74722\n']
['24 30216 74739\n']
['25 30207 74741\n']
['26 30201 74752\n']
['27 30197 74742\n']
['28 30205 74727\n']
['29 30186 74736\n']
['30 30162 74749\n']
['31 30166 74733\n']
['32 30143 74743\n']
['33 30125 74755\n']
['34 30132 74738\n']
['35 30106 74745\n']
['36 30090 74757\n']
['37 30137 74764\n']
['38 30122 74772\n']
['39 30105 74771\n']
['40 30122 74783\n']
['41 30133 74806\n']
['42 30137 74820\n']
['43 30164 74815\n']
['44 30190 74819\n']
['45 30208 74806\n']
['46 30219 74815\n']
['47 30234 74804\n']
['48 30258 74808\n']
[

In [162]:
def initialize(net):
    # define output variables, network and fwResult
    net_info = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
           'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
           'auxiliary':[], 'cost':[]} for (u, v, d) in net.graph.edges(data=True)}
    
    result = {'theta':[], 'z':[]}
    
    # initial all-or-nothing assignment and update link travel time(cost)
    net.all_or_nothing_assignment()
    net.update_linkcost()

    for linkKey, linkVal in network.items():
        linkVal['cost'].append(net.graph[linkKey[0]][linkKey[1]]['weight'])
        linkVal['auxiliary'].append(net.graph[linkKey[0]][linkKey[1]]['object'].vol)
        linkVal['flow'].append(net.graph[linkKey[0]][linkKey[1]]['object'].vol)
    return network, fwResult

@dask.delayed
def BPR(t0, xa, ca, alpha, beta):
    ta = t0*(1+alpha*(xa/ca)**beta)
    return ta

@dask.delayed
def calculateZ(theta, network, SO):
    z = 0
    for linkKey, linkVal in network.items():
        t0 = linkVal['t0']
        ca = linkVal['capa']
        beta = linkVal['beta']
        alpha = linkVal['alpha']
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        
        if SO == False:
            z += integrate.quad(lambda x: BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
        elif SO == True:
            z += list(map(lambda x : x * BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
    return z

@dask.delayed
def lineSearch(network, SO):
    theta = minimize_scalar(lambda x: calculateZ(x, network, SO), bounds = (0,1), method = 'Bounded')
    return theta.x

@dask.delayed
def Frank_Wolfe_Algo(net, net_info, results):
    ## iterations
    iterNum=0
    iteration = True
    while iteration:
        iterNum += 1
        net.all_or_nothing_assignment()
        net.update_linkcost()
    
        # set auxiliary flow using updated link flow
        for linkKey, linkVal in net_info.items():
            linkVal['auxiliary'].append(net.graph[linkKey[0]][linkKey[1]]['object'].vol)
        
        # getting optimal move size (theta)
        theta = lineSearch(network, SO)
        results['theta'].append(theta)
    
        # set link flow (move) based on the theta, auxiliary flow, and link flow of previous iteration
        for linkKey, linkVal in net_info.items():
            aux = linkVal['auxiliary'][-1]
            flow = linkVal['flow'][-1]
            linkVal['flow'].append(flow + theta*(aux-flow))
        
            net.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
            net.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
        
        # update link travel time
        net.update_linkcost()
    
        # calculate objective function value
        z=0
        for linkKey, linkVal in net_info.items():
            linkVal['cost'].append(net.graph[linkKey[0]][linkKey[1]]['weight'])
            totalcost = net.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
            z+=totalcost
        
        results['z'].append(z)        
        
        # convergence test
        if iterNum == 1:
            iteration = True
        else:
            if abs(results['z'][-2] - results['z'][-1]) <= 0.001 or iterNum==3000:
                iteration = False
    return results['z']

In [ ]:
net_info, results = initialize(Philly)

In [165]:
with ProgressBar():
    out = Frank_Wolfe_Algo(net, net_info, results).compute()

[                                        ] | 0% Completed | 876.13 ms


AttributeError: 'DataFrame' object has no attribute 'all_or_nothing_assignment'